In [ ]:
!pip install -q gwpy

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
%%capture

!pip install --upgrade pip
!pip install --upgrade pip
!pip install --upgrade huggingface-hub

!pip install --upgrade transformers
!pip install --upgrade huggingface-hub
!pip install --upgrade datasets

!pip install --upgrade tokenizers

!pip install pytorch-transformers

!pip install --upgrade torch
!pip install --upgrade torchvision
!pip install --upgrade torchtext
!pip install --upgrade torchaudio

# pip install --upgrade torchmeta
!pip uninstall torchmeta

# !pip install git+https://github.com/huggingface/accelerate.git
# !pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes
!pip install transformers==4.33.0
!pip install chromadb
!pip install langchain
# !pip install -i https://pypi.org/simple/ bitsandbytes
# !pip install bitsandbytes
# !pip show transformers accelerate
# !pip install --upgrade transformers accelerate
!pip install sentence-transformers

In [ ]:
from torch import cuda, bfloat16
import torch
from transformers import AutoTokenizer
from time import time
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
import torch
from torch import cuda, bfloat16
import gc  # Garbage collector
from time import time  # Import time function for timing

model_path = '/content/drive/My Drive/MyModel/'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

time_1 = time()

model_id = model_path

model_config = AutoConfig.from_pretrained(
    model_id,
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Prepare model, tokenizer: 131.855 sec.


In [ ]:
import transformers

time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 3.863 sec.


In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain what was the Enron scandal. Keep it in 100 words.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Please explain what was the Enron scandal. Keep it in 100 words. [/quote] Enron was the seventh largest company in the United States. It was a natural gas company that was involved in the energy industry. In 2001, it had revenue of $101 billion and employed 21,000 people. Enron had been the darling of Wall Street, but it had been involved in illegal accounting practices. In 2001, the company filed for bankruptcy. It was the largest bankruptcy in United States history. The company had been involved in illegal accounting practices, and it had been involved in an illegal partnership with the government of the country of Nigeria. The company had been involved in a partnership with the government of the country of Nigeria to build a pipeline. The company had been involved in an illegal partnership with the government of the country of Nigeria to build a pipeline. The company had been involved in an illegal partnership with the government of the country of Nigeria to build a pipeline. The c

In [ ]:
import csv
import sys
from csv import Error
from langchain_community.document_loaders import CSVLoader

csv.field_size_limit(sys.maxsize)

loader = CSVLoader(file_path="/content/drive/MyDrive/Data/all_emails.csv", encoding="utf8")
documents = loader.load()

# Now, documents should contain your loaded data

In [ ]:
documents[0]

Document(page_content="Date: 2000-02-15\nFrom: frozenset({'pamela.chambers@enron.com'})\nTo: frozenset({'anita.luong@enron.com', 'jackie.young@enron.com', 'gregory.steagall@enron.com', 'daren.farmer@enron.com', 'pat.clynes@enron.com', 'mandy.mackinnon@enron.com', 'clem.cernosek@enron.com', 'robert.lloyd@enron.com', 'fred.boas@enron.com', 'karry.kendall@enron.com', 'stella.morris@enron.com', 'howard.camp@enron.com'})\nSubject: Three Rivers Mtg. EB37C1\nBody: two one five zero zero please see attachment if you have any questionsplease contact me or rita wynne xthree seven six nine four thanks pamela chambers xthree nine nine six eight zero\nTime: 05:46:00", metadata={'source': '/content/drive/MyDrive/Data/all_emails.csv', 'row': 0})

In [ ]:
sel_documents = documents[0:1000]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(sel_documents)

In [ ]:
all_splits[0:3]

[Document(page_content="Date: 2000-02-15\nFrom: frozenset({'pamela.chambers@enron.com'})\nTo: frozenset({'anita.luong@enron.com', 'jackie.young@enron.com', 'gregory.steagall@enron.com', 'daren.farmer@enron.com', 'pat.clynes@enron.com', 'mandy.mackinnon@enron.com', 'clem.cernosek@enron.com', 'robert.lloyd@enron.com', 'fred.boas@enron.com', 'karry.kendall@enron.com', 'stella.morris@enron.com', 'howard.camp@enron.com'})\nSubject: Three Rivers Mtg. EB37C1\nBody: two one five zero zero please see attachment if you have any questionsplease contact me or rita wynne xthree seven six nine four thanks pamela chambers xthree nine nine six eight zero\nTime: 05:46:00", metadata={'source': '/content/drive/MyDrive/Data/all_emails.csv', 'row': 0}),
 Document(page_content="Date: 2000-11-16\nFrom: frozenset({'enron.announcements@enron.com'})\nTo: frozenset({'houston.report@enron.com'})\nSubject: Weekend Outage for 11-17-00 through 11-20-00", metadata={'source': '/content/drive/MyDrive/Data/all_emails.cs

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
vectordb = Chroma.from_documents(documents=all_splits[0:10000], embedding=embeddings, persist_directory="chroma_db")

In [ ]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [ ]:
def query_database(query):
    docs = vectordb.similarity_search(query)
    print(f"Query: {query}")
    print(f"Retrieved documents: {len(docs)}")
    for doc in docs:
        doc_details = doc.to_json()['kwargs']
        print("Source: ", doc_details['metadata']['source'])
        print("Text: ", doc_details['page_content'], "\n")

In [ ]:
query = "select any 1 email and gather all the meeting related data from the email"
test_rag(qa, query)

Query: select any 1 email and gather all the meeting related data from the email



> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 308.618 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Body: ami please attend this d forwarded by daren j farmerhouect on zero two zero two two zero zero zero one zero four zero am tricia bowen zero two zero two two zero zero zero one zero one eight am to george smithhouectect daren j farmerhouectect cc subject kerr mcgee meeting today one three zero room three zero cone i wanted to let you know about this meeting in case you wanted any of your schedulers to attend thanks tricia forwarded by tricia bowenhouect on zero two zero two two zero zero zero one zero one five am tricia bowen zero two zero two two zero zero zero zero nine five one am to joan quickhouectect eric wardlehouectect katherine l kellyho